# V2X Communication Fundamentals

This notebook introduces Vehicle-to-Everything (V2X) communication technologies essential for connected and cooperative autonomous driving. We'll explore communication protocols, message types, and build simple V2X applications.

## Learning Objectives
- Understand V2X communication principles and standards
- Implement basic V2V (Vehicle-to-Vehicle) messaging
- Explore cooperative awareness applications
- Analyze communication performance and reliability

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import json
import time
import threading
import queue
import socket
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional
import pandas as pd
from datetime import datetime, timedelta
import hashlib
import random

# Set random seed
np.random.seed(42)
random.seed(42)

print("Libraries imported successfully!")

## 1. V2X Communication Overview

V2X enables vehicles to communicate with:
- **V2V**: Other vehicles
- **V2I**: Infrastructure (traffic lights, road signs)
- **V2P**: Pedestrians and cyclists
- **V2N**: Network services and cloud
- **V2D**: Any connected device

In [ ]:
@dataclass
class BasicSafetyMessage:
    """Basic Safety Message (BSM) structure"""
    vehicle_id: str
    timestamp: float
    latitude: float  # degrees
    longitude: float  # degrees
    elevation: float  # meters
    speed: float  # m/s
    heading: float  # degrees (0-360)
    acceleration: float  # m/s²
    steering_angle: float  # degrees
    brake_status: bool
    turn_signal: str  # 'left', 'right', 'hazard', 'none'
    
    def to_dict(self):
        return asdict(self)
    
    @classmethod
    def from_dict(cls, data):
        return cls(**data)

@dataclass
class CooperativeAwarenessMessage:
    """Cooperative Awareness Message (CAM) - ETSI standard"""
    station_id: int
    generation_time: float
    station_type: str  # 'passenger_car', 'truck', 'bus', 'motorcycle'
    position: Dict[str, float]  # lat, lon, alt
    velocity: Dict[str, float]  # speed, heading
    dimensions: Dict[str, float]  # length, width, height
    confidence: Dict[str, float]  # position, speed confidence
    
    def to_dict(self):
        return asdict(self)

@dataclass
class EmergencyMessage:
    """Emergency/hazard warning message"""
    vehicle_id: str
    timestamp: float
    emergency_type: str  # 'hard_brake', 'accident', 'hazard', 'emergency_vehicle'
    position: Dict[str, float]
    severity: int  # 1-5 scale
    description: str
    ttl: int  # Time to live in seconds
    
    def to_dict(self):
        return asdict(self)

# Communication performance parameters
class V2XParameters:
    """V2X communication parameters"""
    DSRC_RANGE = 1000  # meters
    DSRC_LATENCY = 0.050  # 50ms
    LTE_V2X_RANGE = 5000  # meters
    LTE_V2X_LATENCY = 0.020  # 20ms
    BSM_FREQUENCY = 10  # Hz
    CAM_FREQUENCY = 10  # Hz
    
    # Packet loss rates
    URBAN_PACKET_LOSS = 0.05
    HIGHWAY_PACKET_LOSS = 0.02
    CONGESTED_PACKET_LOSS = 0.15

print("V2X message structures defined!")
print(f"Communication range: {V2XParameters.DSRC_RANGE}m")
print(f"Typical latency: {V2XParameters.DSRC_LATENCY*1000}ms")
print(f"Message frequency: {V2XParameters.BSM_FREQUENCY}Hz")

## 2. Vehicle Simulation for V2X Testing

Let's create a simple vehicle simulation to generate realistic V2X scenarios:

In [ ]:
class Vehicle:
    """Simulated vehicle for V2X communication"""
    
    def __init__(self, vehicle_id, initial_position, initial_heading=0, vehicle_type='passenger_car'):
        self.vehicle_id = vehicle_id
        self.position = initial_position  # [x, y] in meters
        self.heading = initial_heading  # degrees
        self.speed = 0  # m/s
        self.acceleration = 0  # m/s²
        self.steering_angle = 0  # degrees
        self.brake_status = False
        self.turn_signal = 'none'
        self.vehicle_type = vehicle_type
        
        # V2X communication
        self.message_buffer = queue.Queue()
        self.neighbors = {}  # Other vehicles in communication range
        self.last_bsm_time = 0
        
        # Vehicle dimensions (approximate)
        self.dimensions = {
            'passenger_car': {'length': 4.5, 'width': 1.8, 'height': 1.5},
            'truck': {'length': 12.0, 'width': 2.5, 'height': 3.5},
            'bus': {'length': 10.0, 'width': 2.3, 'height': 3.0},
            'motorcycle': {'length': 2.0, 'width': 0.8, 'height': 1.2}
        }
    
    def update_kinematics(self, dt):
        """Update vehicle position and kinematics"""
        # Simple kinematic model
        self.speed += self.acceleration * dt
        self.speed = max(0, self.speed)  # No negative speed
        
        # Update heading based on steering
        if self.speed > 0.1:  # Only turn when moving
            turning_rate = (self.speed / 2.5) * np.tan(np.radians(self.steering_angle))
            self.heading += turning_rate * dt
            self.heading = self.heading % 360
        
        # Update position
        dx = self.speed * np.cos(np.radians(self.heading)) * dt
        dy = self.speed * np.sin(np.radians(self.heading)) * dt
        
        self.position[0] += dx
        self.position[1] += dy
    
    def generate_bsm(self):
        """Generate Basic Safety Message"""
        # Convert position to lat/lon (simplified)
        lat = 37.7749 + self.position[1] / 111000  # Approximate conversion
        lon = -122.4194 + self.position[0] / (111000 * np.cos(np.radians(lat)))
        
        return BasicSafetyMessage(
            vehicle_id=self.vehicle_id,
            timestamp=time.time(),
            latitude=lat,
            longitude=lon,
            elevation=0.0,  # Assume ground level
            speed=self.speed,
            heading=self.heading,
            acceleration=self.acceleration,
            steering_angle=self.steering_angle,
            brake_status=self.brake_status,
            turn_signal=self.turn_signal
        )
    
    def generate_cam(self):
        """Generate Cooperative Awareness Message"""
        lat = 37.7749 + self.position[1] / 111000
        lon = -122.4194 + self.position[0] / (111000 * np.cos(np.radians(lat)))
        
        return CooperativeAwarenessMessage(
            station_id=int(self.vehicle_id.split('_')[1]),
            generation_time=time.time(),
            station_type=self.vehicle_type,
            position={'lat': lat, 'lon': lon, 'alt': 0.0},
            velocity={'speed': self.speed, 'heading': self.heading},
            dimensions=self.dimensions[self.vehicle_type],
            confidence={'position': 0.95, 'speed': 0.90}
        )
    
    def distance_to(self, other_vehicle):
        """Calculate distance to another vehicle"""
        dx = self.position[0] - other_vehicle.position[0]
        dy = self.position[1] - other_vehicle.position[1]
        return np.sqrt(dx**2 + dy**2)
    
    def set_control(self, acceleration=None, steering=None, brake=None, turn_signal=None):
        """Set vehicle control inputs"""
        if acceleration is not None:
            self.acceleration = acceleration
        if steering is not None:
            self.steering_angle = steering
        if brake is not None:
            self.brake_status = brake
        if turn_signal is not None:
            self.turn_signal = turn_signal

# Create test vehicles
vehicles = [
    Vehicle('vehicle_001', [0, 0], 0, 'passenger_car'),
    Vehicle('vehicle_002', [100, 50], 45, 'truck'),
    Vehicle('vehicle_003', [-50, 100], 180, 'passenger_car'),
    Vehicle('vehicle_004', [200, -30], 270, 'bus')
]

# Set some initial movement
vehicles[0].set_control(acceleration=2.0, steering=0)
vehicles[1].set_control(acceleration=1.5, steering=-5)
vehicles[2].set_control(acceleration=3.0, steering=10)
vehicles[3].set_control(acceleration=1.0, steering=0)

print(f"Created {len(vehicles)} test vehicles")
for i, vehicle in enumerate(vehicles):
    print(f"  {vehicle.vehicle_id}: position {vehicle.position}, heading {vehicle.heading}°")

## 3. V2X Communication Simulation

Now let's implement a V2X communication system with realistic constraints:

In [ ]:
class V2XCommunicationSystem:
    """Simulated V2X communication system"""
    
    def __init__(self, environment_type='urban'):
        self.environment_type = environment_type
        self.vehicles = {}
        self.message_log = []
        self.performance_stats = {
            'messages_sent': 0,
            'messages_received': 0,
            'messages_lost': 0,
            'average_latency': 0,
            'latencies': []
        }
        
        # Environment-specific parameters
        self.packet_loss_rate = {
            'urban': V2XParameters.URBAN_PACKET_LOSS,
            'highway': V2XParameters.HIGHWAY_PACKET_LOSS,
            'congested': V2XParameters.CONGESTED_PACKET_LOSS
        }[environment_type]
    
    def register_vehicle(self, vehicle):
        """Register a vehicle in the communication system"""
        self.vehicles[vehicle.vehicle_id] = vehicle
    
    def calculate_propagation_delay(self, distance):
        """Calculate signal propagation delay"""
        speed_of_light = 3e8  # m/s
        propagation_delay = distance / speed_of_light
        processing_delay = V2XParameters.DSRC_LATENCY
        return propagation_delay + processing_delay
    
    def is_in_communication_range(self, sender, receiver):
        """Check if two vehicles are in communication range"""
        distance = sender.distance_to(receiver)
        return distance <= V2XParameters.DSRC_RANGE
    
    def simulate_packet_loss(self):
        """Simulate packet loss based on environment"""
        return np.random.random() < self.packet_loss_rate
    
    def broadcast_message(self, sender_id, message):
        """Broadcast message to vehicles in range"""
        sender = self.vehicles[sender_id]
        broadcast_time = time.time()
        
        self.performance_stats['messages_sent'] += 1
        
        for receiver_id, receiver in self.vehicles.items():
            if receiver_id == sender_id:
                continue  # Don't send to self
            
            if self.is_in_communication_range(sender, receiver):
                # Calculate transmission delay
                distance = sender.distance_to(receiver)
                delay = self.calculate_propagation_delay(distance)
                
                # Simulate packet loss
                if self.simulate_packet_loss():
                    self.performance_stats['messages_lost'] += 1
                    continue
                
                # Successful transmission
                self.performance_stats['messages_received'] += 1
                self.performance_stats['latencies'].append(delay)
                
                # Log message transmission
                log_entry = {
                    'timestamp': broadcast_time,
                    'sender': sender_id,
                    'receiver': receiver_id,
                    'message_type': type(message).__name__,
                    'distance': distance,
                    'latency': delay,
                    'success': True
                }
                self.message_log.append(log_entry)
                
                # Add to receiver's message buffer
                receiver.message_buffer.put({
                    'sender': sender_id,
                    'message': message,
                    'received_time': broadcast_time + delay
                })
    
    def update_communication(self, dt):
        """Update V2X communication for all vehicles"""
        current_time = time.time()
        
        for vehicle in self.vehicles.values():
            # Send BSM at regular intervals
            if current_time - vehicle.last_bsm_time >= (1.0 / V2XParameters.BSM_FREQUENCY):
                bsm = vehicle.generate_bsm()
                self.broadcast_message(vehicle.vehicle_id, bsm)
                vehicle.last_bsm_time = current_time
    
    def get_performance_summary(self):
        """Get communication performance summary"""
        if self.performance_stats['latencies']:
            self.performance_stats['average_latency'] = np.mean(self.performance_stats['latencies'])
        
        total_attempts = self.performance_stats['messages_sent']
        success_rate = self.performance_stats['messages_received'] / max(1, total_attempts)
        loss_rate = self.performance_stats['messages_lost'] / max(1, total_attempts)
        
        return {
            'environment': self.environment_type,
            'total_messages_sent': total_attempts,
            'messages_received': self.performance_stats['messages_received'],
            'messages_lost': self.performance_stats['messages_lost'],
            'success_rate': success_rate,
            'loss_rate': loss_rate,
            'average_latency_ms': self.performance_stats['average_latency'] * 1000,
            'max_latency_ms': max(self.performance_stats['latencies'], default=0) * 1000
        }

# Create communication system
v2x_system = V2XCommunicationSystem('urban')

# Register vehicles
for vehicle in vehicles:
    v2x_system.register_vehicle(vehicle)

print("V2X communication system initialized")
print(f"Environment: {v2x_system.environment_type}")
print(f"Expected packet loss rate: {v2x_system.packet_loss_rate*100:.1f}%")

## 4. Simulation Execution and Visualization

Let's run the V2X simulation and visualize the results:

In [ ]:
def run_simulation(duration=10.0, dt=0.1):
    """Run V2X simulation for specified duration"""
    simulation_data = []
    
    start_time = time.time()
    sim_time = 0
    
    print(f"Running simulation for {duration} seconds...")
    
    while sim_time < duration:
        # Update vehicle kinematics
        for vehicle in vehicles:
            vehicle.update_kinematics(dt)
            
            # Add some randomness to control inputs
            if np.random.random() < 0.1:  # 10% chance to change behavior
                new_accel = vehicle.acceleration + np.random.normal(0, 0.5)
                new_accel = np.clip(new_accel, -5, 3)  # Reasonable limits
                vehicle.set_control(acceleration=new_accel)
        
        # Update V2X communication
        v2x_system.update_communication(dt)
        
        # Record simulation state
        frame_data = {
            'time': sim_time,
            'vehicles': {}
        }
        
        for vehicle in vehicles:
            frame_data['vehicles'][vehicle.vehicle_id] = {
                'position': vehicle.position.copy(),
                'heading': vehicle.heading,
                'speed': vehicle.speed,
                'acceleration': vehicle.acceleration
            }
        
        simulation_data.append(frame_data)
        
        sim_time += dt
        time.sleep(0.01)  # Small delay for realistic timing
    
    print("Simulation completed!")
    return simulation_data

def visualize_simulation(simulation_data):
    """Visualize simulation results"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Extract data for plotting
    times = [frame['time'] for frame in simulation_data]
    
    # Vehicle trajectories
    ax1 = axes[0, 0]
    colors = ['blue', 'red', 'green', 'orange']
    
    for i, vehicle_id in enumerate(vehicles[0].vehicle_id for vehicle in vehicles):
        positions = [frame['vehicles'][f'vehicle_{i+1:03d}']['position'] for frame in simulation_data]
        x_coords = [pos[0] for pos in positions]
        y_coords = [pos[1] for pos in positions]
        
        ax1.plot(x_coords, y_coords, color=colors[i], linewidth=2, 
                label=f'Vehicle {i+1}', alpha=0.8)
        ax1.scatter(x_coords[0], y_coords[0], color=colors[i], s=100, marker='o')
        ax1.scatter(x_coords[-1], y_coords[-1], color=colors[i], s=100, marker='s')
    
    ax1.set_xlabel('X Position (m)')
    ax1.set_ylabel('Y Position (m)')
    ax1.set_title('Vehicle Trajectories')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.axis('equal')
    
    # Speed profiles
    ax2 = axes[0, 1]
    for i, vehicle_id in enumerate(vehicles[0].vehicle_id for vehicle in vehicles):
        speeds = [frame['vehicles'][f'vehicle_{i+1:03d}']['speed'] for frame in simulation_data]
        ax2.plot(times, speeds, color=colors[i], linewidth=2, label=f'Vehicle {i+1}')
    
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Speed (m/s)')
    ax2.set_title('Vehicle Speed Profiles')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Communication statistics
    ax3 = axes[1, 0]
    perf_summary = v2x_system.get_performance_summary()
    
    categories = ['Sent', 'Received', 'Lost']
    values = [perf_summary['total_messages_sent'], 
              perf_summary['messages_received'],
              perf_summary['messages_lost']]
    colors_bar = ['blue', 'green', 'red']
    
    bars = ax3.bar(categories, values, color=colors_bar, alpha=0.7)
    ax3.set_ylabel('Number of Messages')
    ax3.set_title('V2X Communication Statistics')
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                str(value), ha='center', va='bottom')
    
    # Latency distribution
    ax4 = axes[1, 1]
    if v2x_system.performance_stats['latencies']:
        latencies_ms = [lat * 1000 for lat in v2x_system.performance_stats['latencies']]
        ax4.hist(latencies_ms, bins=20, alpha=0.7, color='purple', edgecolor='black')
        ax4.axvline(perf_summary['average_latency_ms'], color='red', linestyle='--', 
                   linewidth=2, label=f'Mean: {perf_summary["average_latency_ms"]:.1f}ms')
    
    ax4.set_xlabel('Latency (ms)')
    ax4.set_ylabel('Frequency')
    ax4.set_title('Communication Latency Distribution')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return perf_summary

# Run simulation
simulation_data = run_simulation(duration=5.0, dt=0.1)
performance_summary = visualize_simulation(simulation_data)

# Print performance summary
print("\nV2X Communication Performance Summary:")
print(f"Environment: {performance_summary['environment']}")
print(f"Total messages sent: {performance_summary['total_messages_sent']}")
print(f"Messages received: {performance_summary['messages_received']}")
print(f"Messages lost: {performance_summary['messages_lost']}")
print(f"Success rate: {performance_summary['success_rate']*100:.1f}%")
print(f"Loss rate: {performance_summary['loss_rate']*100:.1f}%")
print(f"Average latency: {performance_summary['average_latency_ms']:.2f}ms")
print(f"Maximum latency: {performance_summary['max_latency_ms']:.2f}ms")

## 5. Cooperative Driving Applications

Let's implement some practical V2X applications for autonomous driving:

In [ ]:
class CooperativeAwarenessApp:
    """Cooperative awareness application using V2X data"""
    
    def __init__(self, vehicle, communication_system):
        self.vehicle = vehicle
        self.comm_system = communication_system
        self.nearby_vehicles = {}
        self.threat_level = 0  # 0-5 scale
        
    def process_received_messages(self):
        """Process received V2X messages"""
        self.nearby_vehicles.clear()
        
        # Process all messages in buffer
        while not self.vehicle.message_buffer.empty():
            try:
                message_data = self.vehicle.message_buffer.get_nowait()
                sender_id = message_data['sender']
                message = message_data['message']
                
                if isinstance(message, BasicSafetyMessage):
                    # Convert lat/lon back to local coordinates (simplified)
                    x = (message.longitude + 122.4194) * 111000 * np.cos(np.radians(message.latitude))
                    y = (message.latitude - 37.7749) * 111000
                    
                    self.nearby_vehicles[sender_id] = {
                        'position': [x, y],
                        'speed': message.speed,
                        'heading': message.heading,
                        'acceleration': message.acceleration,
                        'brake_status': message.brake_status,
                        'timestamp': message.timestamp
                    }
                    
            except queue.Empty:
                break
    
    def assess_collision_risk(self):
        """Assess collision risk with nearby vehicles"""
        max_threat = 0
        threatening_vehicle = None
        
        for vehicle_id, vehicle_data in self.nearby_vehicles.items():
            # Calculate relative position and velocity
            rel_pos = np.array(vehicle_data['position']) - np.array(self.vehicle.position)
            distance = np.linalg.norm(rel_pos)
            
            if distance > 100:  # Only consider nearby vehicles
                continue
            
            # Simple threat assessment based on distance and relative velocity
            relative_speed = abs(vehicle_data['speed'] - self.vehicle.speed)
            
            # Time to collision estimate (simplified)
            if relative_speed > 1.0:  # Avoid division by near-zero
                time_to_collision = distance / relative_speed
            else:
                time_to_collision = float('inf')
            
            # Threat level calculation
            if time_to_collision < 3.0 and distance < 30:
                threat = 5  # Critical
            elif time_to_collision < 5.0 and distance < 50:
                threat = 3  # High
            elif time_to_collision < 10.0 and distance < 80:
                threat = 1  # Low
            else:
                threat = 0  # No threat
            
            if threat > max_threat:
                max_threat = threat
                threatening_vehicle = vehicle_id
        
        self.threat_level = max_threat
        return max_threat, threatening_vehicle
    
    def generate_warnings(self):
        """Generate safety warnings based on threat assessment"""
        warnings = []
        
        if self.threat_level >= 4:
            warnings.append("CRITICAL: Imminent collision risk detected!")
        elif self.threat_level >= 3:
            warnings.append("WARNING: High collision risk")
        elif self.threat_level >= 1:
            warnings.append("CAUTION: Vehicle in proximity")
        
        # Check for emergency braking situations
        for vehicle_id, vehicle_data in self.nearby_vehicles.items():
            if vehicle_data['brake_status'] and vehicle_data['acceleration'] < -3.0:
                warnings.append(f"Emergency braking detected from {vehicle_id}")
        
        return warnings

class EmergencyBrakeWarning:
    """Emergency Electronic Brake Light (EEBL) application"""
    
    def __init__(self, vehicle, communication_system):
        self.vehicle = vehicle
        self.comm_system = communication_system
        self.emergency_brake_threshold = -4.0  # m/s²
        self.last_emergency_time = 0
    
    def check_emergency_braking(self):
        """Check if vehicle is emergency braking"""
        return (self.vehicle.brake_status and 
                self.vehicle.acceleration <= self.emergency_brake_threshold)
    
    def send_emergency_warning(self):
        """Send emergency brake warning message"""
        if self.check_emergency_braking():
            current_time = time.time()
            
            # Only send warning once per emergency event
            if current_time - self.last_emergency_time > 2.0:
                # Convert position to lat/lon
                lat = 37.7749 + self.vehicle.position[1] / 111000
                lon = -122.4194 + self.vehicle.position[0] / (111000 * np.cos(np.radians(lat)))
                
                emergency_msg = EmergencyMessage(
                    vehicle_id=self.vehicle.vehicle_id,
                    timestamp=current_time,
                    emergency_type='hard_brake',
                    position={'lat': lat, 'lon': lon},
                    severity=4,
                    description='Emergency braking in progress',
                    ttl=30  # 30 seconds
                )
                
                self.comm_system.broadcast_message(self.vehicle.vehicle_id, emergency_msg)
                self.last_emergency_time = current_time
                
                return True
        
        return False

# Create cooperative applications for all vehicles
coop_apps = []
eebl_apps = []

for vehicle in vehicles:
    coop_app = CooperativeAwarenessApp(vehicle, v2x_system)
    eebl_app = EmergencyBrakeWarning(vehicle, v2x_system)
    
    coop_apps.append(coop_app)
    eebl_apps.append(eebl_app)

print(f"Created cooperative applications for {len(vehicles)} vehicles")

# Simulate emergency scenario
def simulate_emergency_scenario():
    """Simulate an emergency braking scenario"""
    print("\nSimulating emergency braking scenario...")
    
    # Reset vehicle positions for clear scenario
    vehicles[0].position = [0, 0]
    vehicles[1].position = [50, 0]  # Following vehicle
    vehicles[0].speed = 20  # 20 m/s (72 km/h)
    vehicles[1].speed = 20
    
    scenario_data = []
    warnings_log = []
    
    for step in range(50):  # 5 seconds at 0.1s intervals
        # At step 20, first vehicle emergency brakes
        if step == 20:
            vehicles[0].set_control(acceleration=-6.0, brake=True)
            print(f"Step {step}: Vehicle 1 emergency braking!")
        
        # Update vehicles
        for vehicle in vehicles[:2]:  # Only use first two vehicles
            vehicle.update_kinematics(0.1)
        
        # Update V2X communication
        v2x_system.update_communication(0.1)
        
        # Process cooperative applications
        for i, (coop_app, eebl_app) in enumerate(zip(coop_apps[:2], eebl_apps[:2])):
            coop_app.process_received_messages()
            
            if eebl_app.send_emergency_warning():
                warnings_log.append(f"Step {step}: Emergency warning sent by Vehicle {i+1}")
            
            threat_level, threatening_vehicle = coop_app.assess_collision_risk()
            warnings = coop_app.generate_warnings()
            
            if warnings:
                for warning in warnings:
                    warnings_log.append(f"Step {step}: Vehicle {i+1} - {warning}")
        
        # Record scenario state
        scenario_data.append({
            'step': step,
            'vehicle_1_pos': vehicles[0].position[0],
            'vehicle_1_speed': vehicles[0].speed,
            'vehicle_2_pos': vehicles[1].position[0],
            'vehicle_2_speed': vehicles[1].speed,
            'distance': abs(vehicles[1].position[0] - vehicles[0].position[0])
        })
        
        time.sleep(0.02)  # Small delay for realistic timing
    
    return scenario_data, warnings_log

# Run emergency scenario
emergency_data, emergency_warnings = simulate_emergency_scenario()

# Visualize emergency scenario
plt.figure(figsize=(15, 10))

# Position and speed plots
steps = [d['step'] for d in emergency_data]
v1_positions = [d['vehicle_1_pos'] for d in emergency_data]
v1_speeds = [d['vehicle_1_speed'] for d in emergency_data]
v2_positions = [d['vehicle_2_pos'] for d in emergency_data]
v2_speeds = [d['vehicle_2_speed'] for d in emergency_data]
distances = [d['distance'] for d in emergency_data]

plt.subplot(2, 2, 1)
plt.plot(steps, v1_positions, 'b-', linewidth=2, label='Vehicle 1 (Lead)')
plt.plot(steps, v2_positions, 'r-', linewidth=2, label='Vehicle 2 (Following)')
plt.axvline(x=20, color='red', linestyle='--', alpha=0.7, label='Emergency Braking')
plt.xlabel('Time Step (0.1s)')
plt.ylabel('Position (m)')
plt.title('Vehicle Positions During Emergency Scenario')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.plot(steps, v1_speeds, 'b-', linewidth=2, label='Vehicle 1 Speed')
plt.plot(steps, v2_speeds, 'r-', linewidth=2, label='Vehicle 2 Speed')
plt.axvline(x=20, color='red', linestyle='--', alpha=0.7, label='Emergency Braking')
plt.xlabel('Time Step (0.1s)')
plt.ylabel('Speed (m/s)')
plt.title('Vehicle Speeds During Emergency Scenario')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.plot(steps, distances, 'g-', linewidth=2, label='Inter-vehicle Distance')
plt.axhline(y=10, color='red', linestyle=':', alpha=0.7, label='Minimum Safe Distance')
plt.axvline(x=20, color='red', linestyle='--', alpha=0.7, label='Emergency Braking')
plt.xlabel('Time Step (0.1s)')
plt.ylabel('Distance (m)')
plt.title('Inter-vehicle Distance')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
# Warning timeline
warning_steps = []
warning_types = []

for warning in emergency_warnings:
    step_str = warning.split(':')[0].replace('Step ', '')
    try:
        step_num = int(step_str)
        warning_steps.append(step_num)
        if 'Emergency warning' in warning:
            warning_types.append('Emergency')
        elif 'CRITICAL' in warning:
            warning_types.append('Critical')
        elif 'WARNING' in warning:
            warning_types.append('Warning')
        else:
            warning_types.append('Caution')
    except ValueError:
        continue

if warning_steps:
    plt.scatter(warning_steps, [1]*len(warning_steps), 
               c=['red' if wt == 'Emergency' else 'orange' if wt == 'Critical' else 'yellow' 
                  for wt in warning_types], s=100, alpha=0.7)

plt.ylim(0.5, 1.5)
plt.xlabel('Time Step (0.1s)')
plt.ylabel('Warnings')
plt.title('V2X Warning Timeline')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print warning log
print("\nEmergency Scenario Warning Log:")
for warning in emergency_warnings:
    print(f"  {warning}")

## 6. Performance Analysis and Next Steps

Let's analyze the V2X system performance and discuss improvements:

In [ ]:
def analyze_v2x_performance():
    """Comprehensive V2X performance analysis"""
    print("V2X Communication Performance Analysis")
    print("=" * 50)
    
    # Communication metrics
    perf = v2x_system.get_performance_summary()
    
    print(f"\n1. Communication Reliability:")
    print(f"   Success Rate: {perf['success_rate']*100:.1f}%")
    print(f"   Packet Loss Rate: {perf['loss_rate']*100:.1f}%")
    print(f"   Expected vs Actual Loss: {v2x_system.packet_loss_rate*100:.1f}% vs {perf['loss_rate']*100:.1f}%")
    
    print(f"\n2. Latency Performance:")
    print(f"   Average Latency: {perf['average_latency_ms']:.2f}ms")
    print(f"   Maximum Latency: {perf['max_latency_ms']:.2f}ms")
    print(f"   Target Latency (DSRC): {V2XParameters.DSRC_LATENCY*1000:.1f}ms")
    
    print(f"\n3. Message Volume:")
    print(f"   Total Messages Sent: {perf['total_messages_sent']}")
    print(f"   Messages Received: {perf['messages_received']}")
    print(f"   Message Rate: {V2XParameters.BSM_FREQUENCY} Hz per vehicle")
    
    # Safety application effectiveness
    print(f"\n4. Safety Application Analysis:")
    print(f"   Emergency Warnings Generated: {len([w for w in emergency_warnings if 'Emergency' in w])}")
    print(f"   Collision Warnings Generated: {len([w for w in emergency_warnings if 'CRITICAL' in w])}")
    
    # Communication range analysis
    if v2x_system.message_log:
        distances = [log['distance'] for log in v2x_system.message_log]
        print(f"\n5. Communication Range:")
        print(f"   Average Communication Distance: {np.mean(distances):.1f}m")
        print(f"   Maximum Communication Distance: {np.max(distances):.1f}m")
        print(f"   DSRC Range Limit: {V2XParameters.DSRC_RANGE}m")
    
    return perf

def improvement_recommendations():
    """Provide recommendations for V2X system improvements"""
    print("\n\nV2X System Improvement Recommendations")
    print("=" * 50)
    
    recommendations = {
        "Communication Reliability": [
            "Implement adaptive transmission power control",
            "Use redundant communication channels (DSRC + 5G)",
            "Add forward error correction for critical messages"
        ],
        "Latency Optimization": [
            "Prioritize safety-critical messages",
            "Implement edge computing for local processing",
            "Use dedicated safety channels"
        ],
        "Security and Privacy": [
            "Implement certificate-based message authentication",
            "Use pseudonym certificates for privacy protection",
            "Add misbehavior detection mechanisms"
        ],
        "Application Enhancement": [
            "Integrate with advanced driver assistance systems",
            "Add predictive collision warning algorithms",
            "Implement cooperative adaptive cruise control"
        ],
        "Scalability": [
            "Implement congestion control mechanisms",
            "Use hierarchical communication architectures",
            "Add dynamic message rate adaptation"
        ]
    }
    
    for category, items in recommendations.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  • {item}")

def future_v2x_trends():
    """Discuss future trends in V2X technology"""
    print("\n\nFuture V2X Technology Trends")
    print("=" * 50)
    
    trends = {
        "5G-V2X Evolution": [
            "Ultra-low latency communications (< 1ms)",
            "Massive machine-type communications",
            "Network slicing for different application types"
        ],
        "AI Integration": [
            "Machine learning for predictive safety",
            "Intelligent traffic management",
            "Autonomous cooperative driving"
        ],
        "Edge Computing": [
            "Distributed processing at road infrastructure",
            "Real-time data fusion and analytics",
            "Reduced dependency on cloud connectivity"
        ],
        "Standardization": [
            "Global harmonization of V2X standards",
            "Interoperability between different systems",
            "Cybersecurity certification frameworks"
        ]
    }
    
    for category, items in trends.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  • {item}")

# Run analysis
performance_analysis = analyze_v2x_performance()
improvement_recommendations()
future_v2x_trends()

print("\n" + "=" * 50)
print("V2X Communication Fundamentals - Complete!")
print("\nKey Takeaways:")
print("✓ V2X enables cooperative awareness and safety applications")
print("✓ Communication reliability and latency are critical factors")
print("✓ Emergency brake warning systems can prevent accidents")
print("✓ Performance depends on environment and traffic conditions")
print("✓ Future developments focus on 5G, AI, and edge computing")

print("\nNext Steps:")
print("• Explore V2I infrastructure communication")
print("• Implement security and authentication protocols")
print("• Study cooperative adaptive cruise control")
print("• Analyze real-world V2X deployment challenges")